# Automatic feature selection with LASSO regression

In this notebook we will learn how LASSO (Least Absolute Shrinkage and Selection Operator) regression works and how it can assist in automatically selecting which variables should be included using a **Cross-Validation** perspective.

#### Start by importing packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets, linear_model
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels
from statsmodels.api import OLS

#### Load dataset and inspect it

Again we're going to use our diabetes dataset. Inspect it again just to remind yourself
what is in it.

In [ ]:
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

feature_names = diabetes.feature_names

print(diabetes['DESCR'])
print(feature_names)

#### Select subset of data

To speed up calculation, we're going to just use the first 150 observations
using numpy slice notation to grab them out of the X, y

In [ ]:
X = X[:150]
y = y[:150]

print(X)

#### Run OLS first (for comparison)

Remember the standard Sklearn model steps:

1. create the model object
2. call the object's fit method.
3. use the fitted model to predict something.
4. assess the predictions.

In [ ]:
# Create linear regression object
model_ols = linear_model.LinearRegression()

# Train the model using the training sets
model_ols.fit(X, y)

# Make predictions using the testing set
y_hat = model_ols.predict(X)

# The coefficients
print("Coefficients: \n", model_ols.coef_)

# The mean squared error
print("Mean squared error:", mean_squared_error(y, y_hat))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination:", r2_score(y, y_hat))


#### Do it again in the econometrics style

Recall that the package statsmodels is closer to the econometrician's way of doing things. We're going to quickly repeat the steps above but with Statsmodels so we can view it in a nice table form.

In [ ]:
x_with_constant = statsmodels.api.add_constant(X)
result = OLS(y, x_with_constant).fit().summary()

print(result)

#### Plot y and y_hat

Let's also plot y and y_hat compared to one of the most important variables, BMI. We'll see both y and y_hat resemble each other.

In [ ]:
# Plot outputs (comparing 1 variable (BMI in column 3) to y and y_hat
plt.scatter(X[:, 3], y, color="black")
plt.scatter(X[:, 3], y_hat, color="blue")

## Switch to LASSO

Now that we've spent all this time setting up our python environment and getting sklearn, it's almost a trivial step in many cases to try out the latest-and-greatest model.

#### Create a LASSO model object

Today's goal, however, is to do Lasso on this same dataset.
To start, lets create a Lasso object. Notice that we are not
setting the alpha/gamma value when we create it.

In [ ]:
model_lasso = Lasso(alpha=1.0, random_state=0, max_iter=10000) # Note, alpha is set by default to 1.0 so we could have omitted it here (though I kept it in to make it clear)
print(model_lasso)

#### Fit the LASSO

Call the lasso.fit() method. 

In [ ]:
model_lasso.fit(X, y)
print(model_lasso)

In [ ]:
y_hat_lasso = model_lasso.predict(X)
print('y_hat_lasso', y_hat_lasso)

#### Plot it too to compare it with the OLS plot from above

What do you see. Is this expected?

In [ ]:
# Plot outputs
plt.scatter(X[:, 3], y, color="black")
plt.scatter(X[:, 3], y_hat_lasso, color="blue")

plt.show()

#### Compare the actual coefficients created

Class question: How are they different? And how are they similar?

In [ ]:
print(model_lasso.coef_)
print(model_ols.coef_)

## Exercise 01

Use a loop to identify the best value of alpha, as measured by r-squared. Discussion question for once you're done: what was the optimal alpha and why does this make sense? How does this compare to OLS? Why is it that way?

In [ ]:
# Exercise work space

# Starter code: keyt parts omitted.
alphas = np.logspace(-5, -0.05, 30)
for SOMETHING in SOMETHING_ELSE:
    model_lasso = Lasso(alpha=alpha, random_state=0, max_iter=10000)
    # LINE OMIITTED
    # LINE OMIITTED
    r2 = r2_score(y, y_hat_lasso)
    print('R2 for alpha ' + str(alpha) + ': ' + str(r2))

## Operationalizing CV with GridSearch

It seems a little weird to be automatically finding the best model. If we were just applying this to the dataset a single time, this would indeed be p-hacking to the extreme. However, showing its performance on UNSEEN data is quite the opposite of p-hacking.

Here, we're going to operationalize our method for finding th ebest model by using GridSearch. We are going to test a variety of different alphas, similar to above. Define them here using numpy logspace:

In [ ]:
alphas = np.logspace(-3, -0.5, 30)
alphas

We are going to be passing this range of tuning parameters to a GridSearch function
that will test which works best when cross-validation methods are applied.
First though, we have to put the alphas into the form the GridSearchCV funciton
Expects, which is a list of dictionaries.

In [ ]:
tuning_parameters = [{'alpha': alphas}]

Recall that CV works by calculating the fit quality of different folds of the training data. Here we will just use 5 folds. GridSearchCV will automatically implement the folding and testing logic.

In [ ]:
n_folds = 5

#### Create the lasso_cv object from the lasso object

Finally, we have all our objects ready to pass to the GridSearchVC function which will Give us back a classifier object. Notice that we're reusing that model_lasso objectg we created above. The difference is that we will be systematically  handing different parameters from the tuning_parameters list into the model_lasso object.

In [ ]:
model_lasso_cv = GridSearchCV(model_lasso, tuning_parameters, cv=n_folds, refit=False)

#### Fit the lasso_cv object

When we call the model_lasso_cv.fit() method, we will iteratively be calling the Lasso.fit() with different permutations of
tuned parameters and then will return the classifier with the best CV fit.

In [ ]:
model_lasso_cv.fit(X, y)

The classifier object now has a variety of diagnostic metrics, reporting back on different folds within the Cross Validation. Take a look at them below.

In [ ]:
print('model_lasso_cv keys returned:', model_lasso_cv.cv_results_.keys())

Some relevant results are as below, which we'll extract and assign to lists.

In [ ]:
scores = model_lasso_cv.cv_results_['mean_test_score']
scores_std = model_lasso_cv.cv_results_['std_test_score']

print('scores', scores)
print('scores_std', scores_std)

## Exercise 02: 

With your table, explore the scores and alphas lists we've created. Identify which alpha is the best, based on the MSE score returned. 

One way to consider doing this would be to create a for loop to iterate through a range(len(scores)): object, reporting the alphas and scores. save the optimal alpha as a new variable called chosen_alpha.

In [ ]:
# Exercise work space

output_dict = {}
for i in OMITTED_CODE:
    output_dict[alphas[i]] = scores[i]
    
chosen_alpha = max(output_dict, key=output_dict.get)

print('best_alpha', chosen_alpha)

#### Use the built-in attributes to get the best alpha

Fortunately, the authors provide a useful  best_params_ attribute.

In [ ]:
print('best_parameters:', model_lasso_cv.best_params_)

Extract the best alpha, which we will use later.

In [ ]:
chosen_alpha = model_lasso_cv.best_params_['alpha']
print('chosen_alpha', chosen_alpha)

#### Rerun LASSO with the best alpha

Now we can rerun a vanilla (no CV) version of Lasso with that specific alpha.
This will return, for instance, a .coef_ list.

In [ ]:
model_lasso_cv_2 = Lasso(alpha=chosen_alpha, random_state=0, max_iter=10000).fit(X, y)

print("coefficients", model_lasso_cv_2.coef_)

Simply looking at the coefficients tells us which are to be included.
Question: How will we know just by looking?

#### Extract the feature names and colum indices of the features that Lasso has selected.

In [ ]:
selected_coefficient_labels = []
selected_coefficient_indices = []
for i in range(len(model_lasso_cv_2.coef_)):
    print('Coefficient', feature_names[i], 'was', model_lasso_cv_2.coef_[i])
    if abs(model_lasso_cv_2.coef_[i]) > 0:
        selected_coefficient_labels.append(feature_names[i])
        selected_coefficient_indices.append(i)

This process led us to the following selected_coefficient_labels:

In [ ]:
print('selected_coefficient_labels', selected_coefficient_labels)

#### Plot the scores versus the alphas

For fun, let's plot the alphas, scores and a confidence range.
What does this show us about the optimal alpha and how it varies with score?

In [ ]:
plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores)

A fun aspect of the k-fold approach is you can get a measure of the std_errors involved. Plot those below. 

In [ ]:
std_error = scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, scores + std_error, 'b--')
plt.semilogx(alphas, scores - std_error, 'b--')

#### Plot the confidence band and the maximum score

alpha=0.2 controls the translucency of the fill color

In [ ]:
plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

plt.ylabel('CV score +/- std error')
plt.xlabel('alpha')
plt.axhline(np.max(scores), linestyle='--', color='.5')
plt.xlim([alphas[0], alphas[-1]])

plt.show()

## Switch back to slides




## Post-LASSO

Finally, now that we have our selected labels, we can use them to select the numpy array columns that we want to use for a post-LASSO run.

In [ ]:
new_x = X[:, selected_coefficient_indices]
new_x = statsmodels.api.add_constant(new_x)
print('new_x', new_x)

Plug this new x matrix into our statsmodels OLS function and print that out.

In [ ]:
result = OLS(y, new_x).fit().summary()
print(result)

## Class discussion

How does the r-squared of this model compare to the one we did at the start of the lecture?

Given the above, how is the LASSO approach better than a vanilla OLS?

Look at the adjusted R-squared. How does that compare across models. In what ways is the adjusted R-squared similar the CV approach?


